<a href="https://colab.research.google.com/github/gaggs/Machine-Learning-Projects/blob/master/CottonDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [3]:
image_size=[224,224]
train_path='/content/drive/My Drive/Colab Notebooks/Datasets/Dataset/train'
valid_path='/content/drive/My Drive/Colab Notebooks/Datasets/Dataset/val'

In [7]:
resnet= ResNet50(include_top=False,input_shape=image_size+[3],weights='imagenet')

In [9]:
for layer in resnet.layers: #for not training on existing weights
    layer.trainable = False

In [10]:
folders=glob('/content/drive/My Drive/Colab Notebooks/Datasets/Dataset/train')


In [11]:
folders

['/content/drive/My Drive/Colab Notebooks/Datasets/Dataset/train']

In [12]:
x=Flatten()(resnet.output)

In [13]:
x

<tf.Tensor 'flatten/Reshape:0' shape=(None, 100352) dtype=float32>

In [14]:
prediction=Dense(len(folders),activation='softmax')(x)

In [16]:
prediction

<tf.Tensor 'dense/Softmax:0' shape=(None, 1) dtype=float32>

In [17]:
model = Model(inputs=resnet.input, outputs=prediction)

In [19]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [20]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [23]:
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Datasets/Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1957 images belonging to 4 classes.


In [24]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Datasets/Dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 36 images belonging to 5 classes.


In [26]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20


InvalidArgumentError: ignored